In [1]:
# project 8

In [2]:
from google.cloud import bigquery
bq = bigquery.Client()
no_cache = bigquery.QueryJobConfig(use_query_cache=False)
%load_ext google.cloud.bigquery

In [3]:
q1 = bq.query(
"""
SELECT geo_id
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE county_name='Dane'
""", job_config = no_cache
)
q1_df = q1.to_dataframe()

In [4]:
#q1
str(q1_df.iloc[0, 0])

'55025'

In [5]:
q2 = bq.query(
"""
SELECT state_fips_code, COUNT(*) AS count
FROM bigquery-public-data.geo_us_boundaries.counties
GROUP BY state_fips_code
ORDER BY count DESC
LIMIT 5
""", job_config = no_cache
)
q2_df = q2.to_dataframe()

In [6]:
#q2
q2_result = {}
for _,row in q2_df.iterrows():
    q2_result[row["state_fips_code"]] = row["count"]
q2_result

{'48': 254, '13': 159, '51': 133, '21': 120, '29': 115}

In [7]:
#q3
tb = 1024 ** 4
price_per_tb = 6.25
q3_result = {}
q3_result["q1"] = price_per_tb * q1.total_bytes_billed/tb
q3_result["q2"] = price_per_tb * q2.total_bytes_billed/tb
q3_result

{'q1': 5.9604644775390625e-05, 'q2': 5.9604644775390625e-05}

In [8]:
dataset_id = f"{bq.project}.p8"
dataset = bigquery.Dataset(dataset_id)
dataset.Location = "US"
dataset = bq.create_dataset(dataset, exists_ok = True)

In [9]:
table_id = f"{bq.project}.p8.hdma"
job_config = bigquery.LoadJobConfig(source_format=bigquery.SourceFormat.PARQUET)
uri = "gs://cs544_p8/hdma-wi-2021.parquet"
load_job = bq.load_table_from_uri(
    uri, table_id, job_config=job_config
)

In [10]:
#q4
[ds.dataset_id for ds in bq.list_datasets(f"{bq.project}")]

['p8']

In [11]:
q5 = bq.query(
"""
SELECT c.county_name AS county_name, COUNT(*) AS count,
FROM {}.p8.hdma AS h
LEFT JOIN bigquery-public-data.geo_us_boundaries.counties AS c
ON h.county_code=c.geo_id
GROUP BY county_name
ORDER BY count DESC
LIMIT 10
""".format(bq.project)
)
q5_df = q5.to_dataframe()

In [12]:
#q5
q5_result = {}
for _, row in q5_df.iterrows():
    q5_result[row["county_name"]] = row["count"]
q5_result

{'Milwaukee': 46570,
 'Dane': 38557,
 'Waukesha': 34159,
 'Brown': 15615,
 'Racine': 13007,
 'Outagamie': 11523,
 'Kenosha': 10744,
 'Washington': 10726,
 'Rock': 9834,
 'Winnebago': 9310}

In [13]:
url = "https://docs.google.com/spreadsheets/d/11UeIBqQylAyNUBsIO54p6WiYJWHayQMfHDbUWq1jGco/"

external_config = bigquery.ExternalConfig("GOOGLE_SHEETS")
external_config.source_uris = [url]
external_config.options.skip_leading_rows = 1
external_config.autodetect = True

table = bigquery.Table(f"{bq.project}.p8.applications")
table.external_data_configuration = external_config

table = bq.create_table(table, exists_ok=True)

In [14]:
q6 = bq.query(
"""
SELECT COUNT(*) as num_applications
FROM {}.p8.applications
WHERE income=150000
""".format(bq.project)
)
q6_df = q6.to_dataframe()

In [15]:
#q6
q6_df.iloc[0, 0]

1

In [16]:
q7 = bq.query(
"""
SELECT counties.county_name AS county_name, COUNT(*) AS count
FROM {}.p8.applications AS apps
JOIN bigquery-public-data.geo_us_boundaries.counties AS counties
ON ST_CONTAINS(counties.county_geom, ST_GEOGPOINT(apps.longitude, apps.latitude))
WHERE counties.state_fips_code = '55'
GROUP BY counties.county_name
""".format(bq.project)
)
q7_df = q7.to_dataframe()

In [17]:
#q7
q7_result = {}
for _, row in q7_df.iterrows():
    q7_result[row["county_name"]] = row["count"]
q7_result

{'Dane': 16,
 'Monroe': 1,
 'Outagamie': 1,
 'Barron': 2,
 'Door': 5,
 'Walworth': 2,
 'Marinette': 2,
 'Oneida': 1,
 'Kewaunee': 1,
 'Brown': 2,
 'Bayfield': 2,
 'Sheboygan': 2,
 'Sauk': 1,
 'Green Lake': 1,
 'Columbia': 1,
 'Jefferson': 1}

In [18]:
%%bigquery
CREATE OR REPLACE MODEL p8.lr
OPTIONS(model_type="LINEAR_REG", INPUT_LABEL_COLS=["loan_amount"])

AS

SELECT loan_amount, income, loan_term
FROM p8.hdma

Query is running:   0%|          |

""


In [19]:
import time
while True:
    models = bq.list_models(f"{bq.project}.p8")
    if any(model.model_id == "lr" for model in models):
        break
    time.sleep(5)

In [20]:
%%bigquery q8_df
SELECT *
FROM ML.EVALUATE(MODEL p8.lr, (
    SELECT loan_amount, income, loan_term
    FROM p8.hdma
))

Query is running:   0%|          |

Downloading:   0%|          |

In [21]:
#q8
q8_df.at[0, "r2_score"]

0.29165346922392066

In [22]:
%%bigquery q9_df
SELECT *
FROM ML.WEIGHTS(MODEL p8.lr)

Query is running:   0%|          |

Downloading:   0%|          |

In [23]:
#q9
q9_df.iloc[0, 1]

0.804631025398982

In [24]:
%%bigquery q10_df
SELECT * 
FROM ML.PREDICT(MODEL p8.lr, (
    SELECT income, 360 as loan_term, loan_amount
    FROM p8.applications
))

Query is running:   0%|          |

Downloading:   0%|          |

In [25]:
#q10
ratio = (q10_df["loan_amount"] > q10_df["predicted_loan_amount"]).mean()
ratio

0.6666666666666666